In [1]:
# Importing required libraries
import pandas as pd

import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.corpus import wordnet as wn

from stop_words import get_stop_words

import re

from sklearn import preprocessing, model_selection, naive_bayes, metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [2]:
df = pd.read_csv("/home/sneha/Documents/Text_Classification/Consumer_Complaints.csv")

/home/sneha/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,6,11,16) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df.head(10)

,Date received,Product,Sub-product,Issue,Sub-issue,Consumer complaint narrative,Company public response,Company,State,ZIP code,Tags,Consumer consent provided?,Submitted via,Date sent to company,Company response to consumer,Timely response?,Consumer disputed?,Complaint ID
0,04/19/2019,Debt collection,Other debt,Attempts to collect debt not owed,Debt is not yours,NaN,Company believes complaint caused principally ...,Perfection Collection,NY,114XX,NaN,NaN,Web,04/19/2019,Closed with explanation,Yes,NaN,3217401
1,04/19/2019,"Credit reporting, credit repair services, or o...",Other personal consumer report,Incorrect information on your report,Public record information inaccurate,NaN,NaN,"Checkr, Inc",SC,296XX,Servicemember,NaN,Web,04/19/2019,Closed with explanation,Yes,NaN,3216965
2,04/19/2019,Debt collection,Medical debt,Attempts to collect debt not owed,Debt was paid,NaN,NaN,"CMRE Financial Services, Inc.",CA,921XX,NaN,NaN,Web,04/19/2019,Closed with explanation,Yes,NaN,3217515
3,04/19/2019,"Credit reporting, credit repair services, or o...",Credit reporting,Credit monitoring or identity theft protection...,Billing dispute for services,NaN,NaN,"Diversified Consultants, Inc.",FL,331XX,NaN,NaN,Web,04/19/2019,In progress,Yes,NaN,3217129
4,04/19/2019,Debt collection,I do not know,Attempts to collect debt not owed,Debt is not yours,NaN,NaN,"Harris & Harris, Ltd.",ME,NaN,NaN,NaN,Web,04/19/2019,Closed with explanation,Yes,NaN,3217139
5,04/19/2019,Mortgage,FHA mortgage,Applying for a mortgage or refinancing an exis...,NaN,NaN,Company believes it acted appropriately as aut...,BAY EQUITY LLC,CA,903XX,NaN,NaN,Web,04/19/2019,Closed with explanation,Yes,NaN,3216932
6,04/19/2019,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,NaN,NaN,"BAYVIEW LOAN SERVICING, LLC",FL,321XX,NaN,NaN,Web,04/19/2019,In progress,Yes,NaN,3217407
7,04/19/2019,Debt collection,Other debt,False statements or representation,Attempted to collect wrong amount,NaN,NaN,"Diversified Consultants, Inc.",FL,33334,NaN,NaN,Web,04/19/2019,In progress,Yes,NaN,3217522
8,04/19/2019,Mortgage,Other type of mortgage,Applying for a mortgage,NaN,NaN,NaN,NAVY FEDERAL CREDIT UNION,FL,329XX,NaN,NaN,Web,04/19/2019,In progress,Yes,NaN,3217456
9,04/19/2019,Mortgage,Conventional home mortgage,Struggling to pay mortgage,NaN,NaN,NaN,"BAYVIEW LOAN SERVICING, LLC",AL,368XX,NaN,NaN,Web,04/19/2019,In progress,Yes,NaN,3216879


In [4]:
# Labels to classify into
df.Product.unique()

array(['Debt collection',
       'Credit reporting, credit repair services, or other personal consumer reports',
       'Mortgage', 'Checking or savings account',
       'Payday loan, title loan, or personal loan', 'Student loan',
       'Money transfer, virtual currency, or money service',
       'Vehicle loan or lease', 'Credit card or prepaid card',
       'Credit reporting', 'Credit card', 'Bank account or service',
       'Consumer Loan', 'Prepaid card', 'Payday loan',
       'Other financial service', 'Money transfers', 'Virtual currency'],
      dtype=object)

In [5]:
df = df[["Consumer complaint narrative", "Product"]]

In [6]:
df.dropna(subset = ['Consumer complaint narrative'], how='all', inplace = True)
len(df)

380614

In [7]:
df.head(10)

,Consumer complaint narrative,Product
19938,XX/XX/XXXX - called Rushmore Loan Management S...,Debt collection
21672,Upon looking at my credit file in XXXX of XXXX...,Debt collection
22595,I paid a medical debt on my credit directly to...,Debt collection
22607,I have been trying to access my Credit Reports...,"Credit reporting, credit repair services, or o..."
22635,XXXX is reporting an account on my credit repo...,Debt collection
22827,I received a letter that said I owe XXXX dolla...,Debt collection
23082,Unlawful foreclosure. I have submitted a full ...,Mortgage
23260,XXXX XXXX bought my mortgage and i can not log...,Mortgage
23273,I borrowed XXXX for school at XXXX XXXX colleg...,Debt collection
23582,False reporting by 2 creditors on my credit re...,"Credit reporting, credit repair services, or o..."


In [8]:
# Train - Test Split
train_x, test_x, train_y, test_y = model_selection.train_test_split(df['Consumer complaint narrative'], df['Product'])

In [9]:
# Label encoding - encode the labels by assigning numerical value to each class
label_encoder = preprocessing.LabelEncoder()
train_y = label_encoder.fit_transform(train_y)
test_y = label_encoder.fit_transform(test_y)

In [10]:
# Converting texts to a matrix of token counts
count_vect = CountVectorizer(analyzer='word', token_pattern=r'\w{1,}')
count_vect.fit(df['Consumer complaint narrative'])

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_accents=None, token_pattern='\\w{1,}', tokenizer=None,
        vocabulary=None)

In [11]:
# Procuring texts-terms matrix (w1,w2) - count
train_count =  count_vect.transform(train_x)
test_count =  count_vect.transform(test_x)

In [12]:
# Fit the model, predict the labels and measure accuracy
fit_naive_bayes = naive_bayes.MultinomialNB().fit(train_count, train_y)
predictions = fit_naive_bayes.predict(test_count)
metrics.accuracy_score(predictions, test_y)

0.6954831115875318

In [13]:
# Alternative
fit_naive_bayes.score(test_count,test_y)

0.6954831115875318

In [15]:
# Converting texts into TF-IDF feature
tfidf_converter = TfidfVectorizer(max_features=5000, norm='l2', stop_words='english')
tfidf_converter.fit(df['Consumer complaint narrative'])

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='latin-1', input='content',
        lowercase=True, max_df=1.0, max_features=1500, min_df=5,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
        stop_words='english', strip_accents=None, sublinear_tf=True,
        token_pattern='(?u)\\b\\w\\w+\\b', tokenizer=None, use_idf=True,
        vocabulary=None)

In [19]:
train_tfidf =  tfidf_converter.transform(train_x)
test_tfidf =  tfidf_converter.transform(test_x)

/home/sneha/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):


In [20]:
# Fit the model, predict the labels and measure accuracy
fit_naive_bayes = naive_bayes.MultinomialNB().fit(train_tfidf, train_y)
predictions = fit_naive_bayes.predict(test_tfidf)
metrics.accuracy_score(predictions, test_y)

0.6647749963217521